BREATH-FIRST SEARCH ALGORITHM IN GRAPHS

In [462]:
import pandas as pd
import numpy as np
from collections import deque
from gdf_output import *

In [463]:
class Graph:
    def __init__(self, archive):
        self.nvertex, self.adjmatrix, self.adjlist = self.openFile(archive)

    def openFile(self, archive):
        with open(archive, 'r') as file:
            nvertex = int(file.readline())
            line = file.readlines()
            adjmatrix = [list(map(lambda x: int(x), l.split())) for l in line[0:]]
            adjlist = [[] for _ in range(nvertex)]

            for i in range(nvertex):
                for j in range(nvertex):
                    if adjmatrix[i][j] == 1:
                        adjlist[i].append(j+1)

        return nvertex, adjmatrix, adjlist

    def printMatrix(self):
        print("Adjacency Matrix:")
        print(np.arange(1,len(self.adjmatrix)+1,1))
        for i, l in enumerate(self.adjmatrix):
            print(f'{i+1} |{" ".join(map(str, l))}')

    def printList(self):
        print("\nAdjacency List:")
        for i, ladj in enumerate(self.adjlist):
            print(f'{i+1}: {"-> ".join(map(str, ladj))}')

In [464]:
class BreathSearch:
    def __init__(self, adj_list, start):
        self.adj_list = adj_list
        self.start = start
        self.result = self.startBFS(self.start)

    def init(self):
        n = int(len(self.adj_list))
        t = 0
        output = {'T': [0] * n, 'LEVEL': [0] * n, 'FATHER': [0] * n}
        return t, n, output

    def BFS(self, v, output, t):
        t += 1
        output['LEVEL'][v-1] = 0
        output['T'][v-1] = t
        visited = []
        visited.append(v)
        queue = deque()
        queue.append(v)
        edges = []
        while queue:
            u = queue.popleft()
            for w in self.adj_list[u-1]:
                if w not in visited:
                    output['FATHER'][w-1] = u
                    output['LEVEL'][w-1] = output['LEVEL'][u-1] + 1
                    t += 1
                    output['T'][w-1] = t
                    visited.append(w)
                    queue.append(w)
                    edges.append((u, w, '0,0,255')) #Father
                else:
                    if u != w and (abs(output['LEVEL'][u-1]-output['LEVEL'][w-1])) < 2:
                        if (w, u) not in [(edge[0], edge[1]) for edge in edges]:
                            if output['FATHER'][u-1] == output['FATHER'][w-1]:
                                edges.append((u, w, '255,0,0'))  # RED - SIMBLINGS
                            elif output['FATHER'][output['FATHER'][u-1]-1] == output['FATHER'][output['FATHER'][w-1]-1]:
                                edges.append((u, w, '255,255,0'))  # YELLOW - COUSIN
                            elif output['LEVEL'][u-1] == output['LEVEL'][w-1]-1 and output['FATHER'][w-1] != u+1:
                                edges.append((u, w, '0,255,0'))  # GREEN - UNCLE


        return t, output, edges

    def startBFS(self, start):
        time, n, output = self.init()
        time, table, edges = self.BFS(self.start, output, time) #table is the main output
        
        for i in range(1, n): 
            if output['T'][i] == 0:
                #print('disconnected graph')
                time, table, edges = self.BFS(i, output, time)
        
        #ORDENANDO AS edges DE FORMA LEXOGRAFICA
        for i, edge in enumerate(edges):
            if edge[0] > edge[1]:
                # Troca os elementos da aresta
                edges[i] = (edge[1], edge[0], edge[2])
        Edges_sorted = sorted(edges, key=lambda x: (x[0], x[1]))

        table = {'n': n, 'vParent': table['FATHER'], 'vchild': range(1, n+1),
                  'T': table['T'], 'LEVEL': table['LEVEL'], 'Edges': Edges_sorted}
        
        
        
        return table

    def printBFS(self, table):
        df = pd.DataFrame({"Vertex": table['vchild'], "Parent": table['vParent'], "T": table['T'], 'LEVEL': table['LEVEL']})
        dfOrd = df.sort_values(by='Vertex')
        print("\n RESULTS: ----------------------------------------")
        print(dfOrd.to_string(index=False))

        print("\nEdges:")
        for edge in table['Edges']:
            print(edge)



In [465]:
#Lendo o grafo e imprimindo o grafo
archive = 'graph01.txt'
graph = Graph(archive)
graph.printMatrix()
graph.printList()

Adjacency Matrix:
[ 1  2  3  4  5  6  7  8  9 10]
1 |0 0 1 0 0 0 0 0 0 0
2 |0 0 0 1 0 1 0 0 1 0
3 |1 0 0 0 0 1 0 0 0 1
4 |0 1 0 0 0 0 0 0 0 0
5 |0 0 0 0 0 1 1 0 0 0
6 |0 1 1 0 1 0 0 0 0 0
7 |0 0 0 0 1 0 0 0 0 0
8 |0 0 0 0 0 0 0 0 0 1
9 |0 1 0 0 0 0 0 0 0 0
10 |0 0 1 0 0 0 0 1 0 0

Adjacency List:
1: 3
2: 4-> 6-> 9
3: 1-> 6-> 10
4: 2
5: 6-> 7
6: 2-> 3-> 5
7: 5
8: 10
9: 2
10: 3-> 8


In [466]:
vstart = 1
bfs =  BreathSearch(graph.adjlist, vstart)
# Chamando o método para iniciar a busca em profundidade
bfs_res = bfs.result
bfs.printBFS(bfs_res)


 RESULTS: ----------------------------------------
 Vertex  Parent  T  LEVEL
      1       0  1      0
      2       6  5      3
      3       1  2      1
      4       2  8      4
      5       6  6      3
      6       3  3      2
      7       5 10      4
      8      10  7      3
      9       2  9      4
     10       3  4      2

Edges:
(1, 3, '0,0,255')
(2, 4, '0,0,255')
(2, 6, '0,0,255')
(2, 9, '0,0,255')
(3, 6, '0,0,255')
(3, 10, '0,0,255')
(5, 6, '0,0,255')
(5, 7, '0,0,255')
(8, 10, '0,0,255')


In [467]:
result = "graph01_bfs.gdf"
gerarGDF(bfs_res, result, 0)

METRICS

In [468]:
 # Eccentricity is the distance to the furthest vertex
     #(how far can I walk with minimum paths?)
     # Central vertices have low eccentricity
     # Peripheral vertices have high eccentricity
     # Radius is the smallest eccentricity of the graph
     # Diameter is the maximum eccentricity of the graph (for this I need the distance between them all)

def metricas(adjlist):

    qtd_Vertex = len(adjlist)
    
    eccentricity = []
    mean = []
    print(qtd_Vertex)
    for i in range(1,qtd_Vertex+1):
        bfs = BreathSearch(adjlist, i)# self.startBFS(i) 
        result = bfs.result
        eccentricity.append(max(result['LEVEL']))
        mean.append(np.mean(result['LEVEL'])) 
    
    #radius
    radius  = (min(eccentricity))
    #diameter
    diameter  = (max(eccentricity))
    #mean
    mean  = (np.mean(mean))
    #Para imprimir:
    df = pd.DataFrame({"Vertex": range(1, qtd_Vertex+1), "Eccentricity": eccentricity})
    dfOrd = df.sort_values(by='Vertex')
    print("\n RESULTS: ----------------------------------------")
    print(dfOrd)

    print("\n")
    print("G radius:", radius)
    print("G diameter:", diameter)
    print("G mean distances:", mean)

    return eccentricity, radius, diameter, mean


In [469]:
metricas(graph.adjlist)

10

 RESULTS: ----------------------------------------
   Vertex  Eccentricity
0       1             4
1       2             4
2       3             3
3       4             5
4       5             4
5       6             3
6       7             5
7       8             5
8       9             5
9      10             4


G radius: 3
G diameter: 5
G mean distances: 2.44


([4, 4, 3, 5, 4, 3, 5, 5, 5, 4], 3, 5, 2.44)

TESTE 2


In [470]:
#Lendo o grafo e imprimindo o grafo
archive2 = 'graph02.txt'
graph2 = Graph(archive2)
graph2.printMatrix()
graph2.printList()
vstart = 1
bfs2=  BreathSearch(graph2.adjlist, vstart)
# Chamando o método para iniciar a busca em profundidade
bfs_res2 = bfs2.result
bfs.printBFS(bfs_res2)
result2 = "graph02_bfs.gdf"
gerarGDF(bfs_res2, result2, 0)

Adjacency Matrix:
[ 1  2  3  4  5  6  7  8  9 10]
1 |0 0 1 0 1 0 1 0 0 0
2 |0 0 0 0 0 1 0 1 0 0
3 |1 0 0 1 1 1 0 0 0 1
4 |0 0 1 0 1 0 0 0 0 0
5 |1 0 1 1 0 0 0 0 0 0
6 |0 1 1 0 0 0 0 0 0 0
7 |1 0 0 0 0 0 0 1 0 0
8 |0 1 0 0 0 0 1 0 1 0
9 |0 0 0 0 0 0 0 1 0 1
10 |0 0 1 0 0 0 0 0 1 0

Adjacency List:
1: 3-> 5-> 7
2: 6-> 8
3: 1-> 4-> 5-> 6-> 10
4: 3-> 5
5: 1-> 3-> 4
6: 2-> 3
7: 1-> 8
8: 2-> 7-> 9
9: 8-> 10
10: 3-> 9

 RESULTS: ----------------------------------------
 Vertex  Parent  T  LEVEL
      1       0  1      0
      2       6  9      3
      3       1  2      1
      4       3  5      2
      5       1  3      1
      6       3  6      2
      7       1  4      1
      8       7  8      2
      9      10 10      3
     10       3  7      2

Edges:
(1, 3, '0,0,255')
(1, 5, '0,0,255')
(1, 7, '0,0,255')
(2, 6, '0,0,255')
(2, 8, '0,255,0')
(3, 4, '0,0,255')
(3, 5, '255,0,0')
(3, 6, '0,0,255')
(3, 10, '0,0,255')
(4, 5, '0,255,0')
(7, 8, '0,0,255')
(8, 9, '0,255,0')
(9, 10, '0,0,255')
